# Wikipedia Article Talk Analysis

This notebook analyzes interactions in Wikipedia article talk pages, focusing on user discussions and contributions. The analysis includes:

1. Processing and cleaning Wikipedia talk page data
2. Analyzing user interactions and contribution patterns
3. Building social and activity graphs from the data
4. Examining temporal patterns in user activities

The analysis uses data from the [wiki-meta dataset](https://snap.stanford.edu/data/wiki-meta.html), which includes both user-talk and article-talk interactions.

## Setup and Dependencies

First, we import the required libraries:
- `pandas` and `numpy` for data manipulation
- `seaborn` and `matplotlib` for visualization
- `networkx` for graph analysis

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pyvis.network import Network
from tqdm.notebook import tqdm
from networkx import bipartite
import itertools
from dask import dataframe as dd

import networkx as nx

sns.set_theme(
    context="notebook",
    style="whitegrid",
    palette="colorblind",
    rc={"text.usetex": False},
)

## Data Overview and Initial Processing

### Dataset Statistics:

1. **User-talk dataset**:
   - Total entries: 11,629,578
   - Bot-generated traffic: 1,600,129 (≈14%)
   - Clean entries: 10,029,449
   - Note: Self-links need to be removed as they don't represent inter-user interactions

2. **Article-talk dataset**:
   - Total entries: 13,384,702
   - Bot-generated traffic:  1,511,005 (≈11%)
   - Clean entries: 11,873,697

## Data Loading and Preprocessing

We load the processed article talk data and elections data, converting timestamps to datetime format for temporal analysis.

In [ ]:
def load_article_edits():
    article_data = []
    for chunk in tqdm(pd.read_csv("data/article_edits.csv", chunksize=1000000)):
        chunk["timestamp"] = (
            pd.to_datetime(chunk["timestamp"]).values.astype(np.int64) // 10**9
        )
        chunk["timestamp"] = pd.to_datetime(chunk["timestamp"], unit="s")

        chunk["minor"] = chunk["minor"].astype(bool)

        article_data.append(chunk)

    article_data = pd.concat(article_data, ignore_index=True)
    article_data["user"] = article_data["user"].astype("category")
    article_data["namespace"] = article_data["namespace"].astype("category")

    return article_data

article_edits = load_article_edits()

voting_data = pd.read_csv("data/nominations_elections.csv")
voting_data["close_time"] = pd.to_datetime(voting_data["close_time"])

0it [00:00, ?it/s]

## Time-based Data Splitting

We split the data into training and test sets based on temporal boundaries:
- Validation cutoff: Last election before 2007-01-01
- Test cutoff: Last election before 2007-12-01

This temporal split helps evaluate our analysis on future data.

In [5]:
val_lim = voting_data[voting_data["close_time"] < "2007-01-01"]["close_time"].max()
test_lim = voting_data[voting_data["close_time"] < "2007-12-01"]["close_time"].max()

train: pd.DataFrame = article_edits[article_edits["timestamp"] <= test_lim]
test: pd.DataFrame = article_edits[article_edits["timestamp"] > test_lim]

NameError: name 'article_edits' is not defined

## Visualization Functions

### Distribution and Box Plot Function

This function creates a combined view of data distribution:
1. Histogram with density estimation (left)
2. Box plot showing quartiles and outliers (right)

Both plots use logarithmic scaling for better visualization of skewed distributions.

In [ ]:
def plot_distribution_and_boxplot(
    data: pd.DataFrame | pd.Series, column: str | None, title: str
):
    """
    Create a distribution plot and boxplot for a specified column in a DataFrame or Series.

    Args:
        data: pandas DataFrame or Series containing the data
        column: name of the column to visualize (None if data is Series)
        title: custom title/name for the plots
    """
    plt.figure(figsize=(12, 6))

    values = data[column] if isinstance(data, pd.DataFrame) else data

    plt.subplot(121)
    sns.histplot(data=values, bins=50, stat="density", log_scale=(True, False))
    plt.title(f"Distribution of {title}")
    plt.xlabel(f"{title} (log scale)".capitalize())
    plt.ylabel("Frequency")

    plt.subplot(122)
    sns.boxplot(y=values)
    plt.yscale("log")
    plt.title(f"Box plot of {title}")
    plt.ylabel(f"{title} (log scale)".capitalize())

    plt.tight_layout()

## Data Analysis

### Revision Word Count Analysis
Analyzing the distribution of word counts in article talk page revisions

In [ ]:
plot_distribution_and_boxplot(train, "textdata", "revision word count")

### Article Interaction Analysis
Examining the distribution of interactions across different article namespaces

In [ ]:
articles = train.groupby(["namespace"]).size().reset_index(name="count")
plot_distribution_and_boxplot(articles, "count", "number of article interactions")

### Election Timing Analysis
Analyzing the temporal patterns of elections:

In [ ]:
intervals = voting_data["close_time"].sort_values()
intervals = (intervals.diff().dt.total_seconds() / (24 * 60 * 60)).dropna()
intervals = intervals.reset_index()
plot_distribution_and_boxplot(
    intervals, column="close_time", title="time between elections (days)"
)

## Graph Building Functions

### Social Graph Construction
Building user interaction networks based on talk page activities:

## Graph Weight Computation

The weight of edges in our interaction graphs is computed using a formula that combines multiple factors to capture the significance and recency of user interactions. Here's the detailed breakdown of the formula:

$$
w_{u,a} = \sum_{r\in\mathcal R_{u\to a}} \log(1+\text{words}_r)\,\bigl(1-0.25\,\mathbb I\{\text{minor}_r\}\bigr)\,e^{-\lambda (t_\text{cutoff}-t_r)}
$$

Where:

1. $w_{u,a}$ is the total weight of interactions between user $u$ and article/namespace $a$
2. $\mathcal R_{u\to a}$ represents all revisions by user $u$ to article/namespace $a$
3. For each revision $r$:
   - $\text{words}_r$ is the word count of the revision
   - $\mathbb I\{\text{minor}_r\}$ is an indicator function (1 if minor edit, 0 if major)
   - $t_r$ is the timestamp of the revision
   - $t_\text{cutoff}$ is our analysis cutoff date
   - $\lambda$ is the decay parameter (set to $\ln(2)/30$ for a 30-day half-life)

### Components of the Formula:

1. **Content Weight**: $\log(1+\text{words}_r)$
   - Uses log-scale to prevent very long revisions from dominating
   - Adding 1 prevents taking log(0) for empty revisions

2. **Edit Type Weight**: $1-0.75\,\mathbb I\{\text{minor}_r\}$
   - Major edits have weight 1.0
   - Minor edits have weight 0.25 (75% reduction)

3. **Time Decay**: $e^{-\lambda (t_\text{cutoff}-t_r)}$
   - Exponential decay based on time difference from cutoff
   - $\lambda = \ln(2)/30$ gives a 30-day half-life
   - More recent interactions have higher weights

### Implementation

The formula is implemented in the code below, where we:
1. Calculate weights for each revision
2. Group by user and namespace
3. Sum the weights to get the final edge weights

In [ ]:
lamb = np.log(2) / 30
train["weight"] = (
    np.log(train["textdata"] + 1)
    * (1 - 0.25 * train["minor"])
    * np.exp(-lamb * (train["timestamp"].rsub(test_lim).dt.days))
)
graph_data = (
    train.groupby(["user", "namespace"])["weight"]
    .sum()
    .reset_index(name="weight")
    .sort_values("weight", ascending=False)
)
graph_data

### Impact of Parameters

- The 30-day half-life ($\lambda = \ln(2)/30$) means that:
  - A 30-day old revision has 50% of its original weight
  - A 60-day old revision has 25% of its original weight
  - A 90-day old revision has 12.5% of its original weight

- The 0.75 reduction for minor edits means:
  - Major edits retain full weight (multiplied by 1.0)
  - Minor edits retain only 25% weight (multiplied by 0.25)

This weighting scheme emphasizes:
1. Substantial contributions (through word count)
2. Major edits over minor ones
3. Recent activity over older interactions

In [14]:
def create_graph_data(
    df: pd.DataFrame,
    cutoff_date: pd.Timestamp,
    users: list[str] | None = None,
    time_window: float = 30,
) -> pd.DataFrame:
    """
    Create graph data from article talk data up to a cutoff date.
    
    Args:
        df: input DataFrame containing article talk data
        cutoff_date: timestamp to filter the data
        users: optional list of usernames to keep in the final output
        time_window: time window in days for exponential decay (default: 30)
    
    Returns:
        DataFrame with user, namespace and weight columns
        
    Raises:
        ValueError: if required columns are missing or time_window is not positive
    """
    required_cols = {"user", "namespace", "timestamp", "minor", "textdata"}
    if not required_cols.issubset(df.columns):
        missing = required_cols - set(df.columns)
        raise ValueError(f"Missing required columns: {missing}")
        
    if time_window <= 0:
        raise ValueError("time_window must be positive")

    # Filter data up to cutoff date
    filtered_data = df[df["timestamp"] <= cutoff_date].copy()
    
    if users is not None:
        filtered_data = filtered_data[filtered_data["user"].isin(users)]

    # Calculate weights using exponential decay
    lamb = np.log(2) / time_window
    filtered_data["weight"] = (
        np.log(filtered_data["textdata"] + 1)
        * (1 - 0.25 * filtered_data["minor"])
        * np.exp(-lamb * (filtered_data["timestamp"].rsub(cutoff_date).dt.days))
    )
    
    return (
        filtered_data.groupby(["user", "namespace"], observed=True)["weight"]
        .sum()
        .reset_index(name="weight")
        .sort_values("weight", ascending=False)
    )


## Graph building

In [ ]:
graph_data = create_graph_data(
    train,
    cutoff_date=test_lim
)

In [57]:
def bipartite_graph_from_edgelist(
    edgelist: pd.DataFrame,
    source: str = "namespace",
    target: str = "user",
    weight: str = "weight",
    B: nx.DiGraph | None = None,
) -> nx.DiGraph:
    """
    Create a bipartite graph from an edge list DataFrame efficiently.

    Args:
        edgelist: DataFrame containing the edge list with user, namespace and weight columns
        source: name of the source column (default: 'namespace')
        target: name of the target column (default: 'user')
        weight: name of the weight column (default: 'weight')

    Returns:
        A NetworkX bipartite graph
    """
    if B is None:
        # Create a new directed graph
        B = nx.DiGraph()

    # Add nodes in bulk operations
    B.add_nodes_from(edgelist[source].unique(), bipartite=0)
    B.add_nodes_from(edgelist[target].unique(), bipartite=1)

    # Add edges in bulk operation
    edges = list(
        zip(
            edgelist[source],
            edgelist[target],
            edgelist[weight],
        )
    )
    B.add_weighted_edges_from(edges)

    return B

In [ ]:
graph = bipartite_graph_from_edgelist(graph_data)

## Interactive Graph Visualization

We use pyvis to create interactive visualizations of our Wikipedia interaction networks. The visualization features:

### Node Properties:
- Size proportional to node degree (larger = more connections)
- Color coding for bipartite sets:
  - Blue: Article namespaces
  - Red: Users

### Edge Properties:
- Directed edges showing edit relationships
- Edge weights reflecting interaction strength

### Interactive Features:
- Zoom and pan
- Node dragging
- Physics simulation controls
- Node/edge filtering options

This visualization helps us understand:
- Community structure
- Key users and articles
- Interaction patterns
- Network density and connectivity

In [ ]:
def visualize_graph(graph: nx.DiGraph):
    """
    Create an interactive visualization of the graph using pyvis.
    
    Args:
        graph: NetworkX graph to visualize
        
    The visualization will:
    - Use a dark theme with white text
    - Show interactive controls for physics, nodes, and edges
    - Scale node sizes based on degree
    - Color nodes based on bipartite sets
    """
    # Create network with custom settings
    network = Network(
        height="1080px", 
        directed=True,
        bgcolor="#222222",
        font_color="white",
        select_menu=True
    )
    
    # Enable interactive controls
    network.show_buttons(filter_=["physics", "nodes", "edges"])
    
    # Add nodes with different colors for bipartite sets
    for node, nodedata in tqdm(graph.nodes(data='bipartite', default=0), desc="Adding nodes"):
        color = "#4287f5" if nodedata == 0 else "#f54242"
        degree = graph.out_degree(node) if nodedata == 0 else graph.in_degree(node)
        network.add_node(
            node,
            color=color,
            size=10 + degree,
            title=f"Node: {node}\nDegree: {degree}"
        )
    
    # Add edges with weights
    for source, target, weight in tqdm(graph.edges.data('weight', default=1.0), desc="Adding edges"):
        network.add_edge(source, target, value=weight)
    
    # Show the graph
    network.show("./graphs/vis/user-article.html")

In [ ]:
visualize_graph(graph)

In [43]:
rev_graph = graph.reverse()

In [44]:
user_graph = bipartite.projected_graph(rev_graph, graph_data["user"].unique()).to_undirected()

In [45]:
user_graph.remove_nodes_from(
    [node for node, degree in user_graph.degree() if degree == 0]
)
user_graph.number_of_nodes(), user_graph.number_of_edges()

(25286, 81927)

## Feature engineering

## Graph Weight Function: Implementation Details

The weight function implementation includes several optimizations for performance:

1. **Efficient Data Structures**:
   - Pre-computed weight dictionary to avoid redundant lookups
   - Set operations for neighborhood comparisons
   - Vectorized numpy operations where possible

2. **Modular Design**:
   - Separate functions for each metric type
   - Reusable components for different graph analyses
   - Clear separation of concerns

3. **Memory Optimization**:
   - Single-pass computations where possible
   - Early returns for edge cases
   - Minimal data duplication

The implementation balances computational efficiency with memory usage, making it suitable for large-scale network analysis.

In [10]:
def compute_basic_weight(common_neighbors: set, nbr_weights: dict) -> float:
    """Compute the basic weight between two nodes based on common neighbors."""
    if not common_neighbors:  # Early return for empty set
        return 0.0
        
    # Pre-calculate values for common neighbors
    weights = [(nbr_weights[nbr]['u'] + nbr_weights[nbr]['v'], 
               nbr_weights[nbr]['total']) for nbr in common_neighbors]
    
    # Use numpy for faster array operations
    weights = np.array(weights)
    return np.sum(weights[:, 0]) / np.sum(weights[:, 1]) if weights.size > 0 else 0.0

def compute_weighted_jaccard(all_neighbors: set, nbr_weights: dict) -> float:
    """Compute the weighted Jaccard similarity between two nodes."""
    if not all_neighbors:  # Early return for empty set
        return 0.0
    
    # Pre-calculate all values in single pass
    weights = [(nbr_weights[nbr]['u'], nbr_weights[nbr]['v']) for nbr in all_neighbors]
    
    # Convert to numpy array for faster operations
    weights = np.array(weights)
    if weights.size == 0:
        return 0.0
        
    intersection = np.minimum(weights[:, 0], weights[:, 1]).sum()
    union = np.maximum(weights[:, 0], weights[:, 1]).sum()
    
    return intersection / union if union != 0 else 0.0

def compute_entropy_and_mutual_info(all_neighbors: set, nbr_weights: dict, total_weight: float) -> tuple[float, float]:
    """
    Compute both participation entropy and mutual information efficiently.
    
    Args:
        all_neighbors: set of all neighbors for both nodes
        nbr_weights: dictionary containing edge weights for each neighbor
        total_weight: sum of weights for both nodes
        
    Returns:
        tuple: (participation_entropy, mutual_information)
    """
    joint_entropy = 0.0
    mutual_info = 0.0
    
    for nbr in all_neighbors:
        nbr_total = nbr_weights[nbr]['total']
        p_u = nbr_weights[nbr]['u'] / nbr_total if nbr_total > 0 else 0
        p_v = nbr_weights[nbr]['v'] / nbr_total if nbr_total > 0 else 0
        
        # Compute entropy
        if p_u > 0:
            joint_entropy += -p_u * np.log2(p_u)
        if p_v > 0:
            joint_entropy += -p_v * np.log2(p_v)
            
        # Compute mutual information
        if total_weight > 0:
            joint_prob = (nbr_weights[nbr]['u'] + nbr_weights[nbr]['v']) / total_weight
            if p_u > 0 and p_v > 0 and joint_prob > 0:
                mutual_info += joint_prob * np.log2(joint_prob / (p_u * p_v))
    
    return joint_entropy, mutual_info

def weight_function(B: nx.DiGraph, u: str, v: str) -> tuple[float, float, float, float]:
    """
    Calculate multiple metrics between two users in a user-article bipartite graph.
    
    Args:
        B: bipartite graph
        u: first user
        v: second user
        
    Returns:
        tuple: (basic_weight, weighted_jaccard, participation_entropy, mutual_info)
    """
    # Get neighborhoods once
    unbrs = set(B.predecessors(u))
    vnbrs = set(B.predecessors(v))
    common_neighbors = unbrs & vnbrs
    all_neighbors = unbrs | vnbrs
    
    # Pre-compute all weights in a single dictionary
    nbr_weights = {
        nbr: {
            'u': B.edges[nbr, u]['weight'] if B.has_edge(nbr, u) else 0.0,
            'v': B.edges[nbr, v]['weight'] if B.has_edge(nbr, v) else 0.0,
            'total': B.out_degree(nbr, weight="weight")
        }
        for nbr in all_neighbors
    }
    
    # Calculate total weight once
    total_weight = B.in_degree(u, weight="weight") + B.in_degree(v, weight="weight")
    
    # Compute all metrics using the pre-computed weights
    basic_weight = compute_basic_weight(common_neighbors, nbr_weights)
    weighted_jaccard = compute_weighted_jaccard(all_neighbors, nbr_weights)
    participation_entropy, mutual_info = compute_entropy_and_mutual_info(
        all_neighbors, nbr_weights, total_weight
    )
    
    return basic_weight, weighted_jaccard, participation_entropy, mutual_info


## Feature Engineering: Graph-Based Metrics

This section implements various graph-based metrics to analyze user interactions in Wikipedia. We compute several important metrics:

### User-Article (Bipartite) Graph Metrics:
1. **Basic Weight**: Normalized sum of weights for common article interactions
2. **Weighted Jaccard**: Similarity measure based on shared article interactions
3. **Participation Entropy**: Measures diversity of user participation
4. **Mutual Information**: Quantifies information shared between user interaction patterns

### User-User (Social) Graph Metrics:
1. **Jaccard Index**: Measures similarity of users' neighbor sets
2. **Adamic-Adar Index**: Weighted similarity metric favoring rare shared neighbors
3. **Preferential Attachment**: Product of node degrees
4. **PageRank**: Measures global importance of users in the network

These metrics help us understand:
- How similarly users interact with articles
- The strength and nature of user relationships
- Users' roles and importance in the Wikipedia community
- Potential voting patterns in admin elections

In [ ]:
def compute_user_article_metrics(B: nx.DiGraph, voters: list[str], candidate: str) -> pd.DataFrame:
    """
    Efficiently compute weight metrics for all pairs of users using numpy vectorization.
    
    Args:
        B: bipartite graph containing user-article interactions
        usernames: list of usernames to analyze
        
    Returns:
        DataFrame containing weight metrics for each user pair
    """
    # Filter valid users that exist in the graph
    valid_voters = [u for u in sorted(voters) if u in B]
    valid_candidate = candidate if candidate in B else None

    if valid_candidate is None:
        return pd.DataFrame(columns=["user1", "user2", "collaboration", "pairwise_jaccard", "participation_entropy", "mutual_information"])

    # Create user pairs using combinations
    pairs = list(itertools.product(valid_voters, [valid_candidate]))
    
    if not pairs:
        return pd.DataFrame(columns=["user1", "user2", "collaboration", "pairwise_jaccard", "participation_entropy", "mutual_information"])
    
    # Pre-allocate metrics list
    metrics = []
    for user1, user2 in pairs:
        # Compute metrics for each pair
        basic_w, jaccard_w, entropy, mutual_info = weight_function(B, user1, user2)

        metrics.append({
        'user1': user1,
        'user2': user2,
        'collaboration': basic_w,
        'pairwise_jaccard': jaccard_w,
        'participation_entropy': entropy,
        'mutual_information': mutual_info
    })
    
    # Create DataFrame
    results = pd.DataFrame(metrics)
    
    # Sort only once at the end
    return results.sort_values('collaboration', ascending=False)

In [47]:
def compute_user_user_metrics(G: nx.Graph, voters: list[str], candidate: str) -> pd.DataFrame:
    """
    Compute graph metrics for pairs of users in a social network using vectorized operations.

    Args:
        G: NetworkX undirected graph representing user interactions
        usernames: List of usernames to analyze

    Returns:
        DataFrame with metrics for each user pair:
        - Jaccard index
        - Adamic-Adar index
        - Preferential attachment
        - PageRank of each node
        - Katz centrality of each node
    """
    # Filter valid users that exist in the graph
    valid_voters = [u for u in sorted(voters) if u in G]
    valid_candidate = candidate if candidate in G else None

    if valid_candidate is None:
        return pd.DataFrame(columns=["user1", "user2", "jaccard", "adamic_adar", "pref_attachment", "pagerank_1", "pagerank_2"])

    # Create user pairs using combinations
    pairs = list(itertools.product(valid_voters, [valid_candidate]))
    
    if not pairs:
        return pd.DataFrame(columns=["user1", "user2", "jaccard", "adamic_adar", "pref_attachment", "pagerank_1", "pagerank_2"])

    # Pre-compute centrality measures
    pagerank = nx.pagerank(G)
    jaccard = {(u, v): p for u, v, p in nx.jaccard_coefficient(G, ebunch=pairs)}
    adamic_adar = {(u, v): p for u, v, p in nx.adamic_adar_index(G, ebunch=pairs)}
    pref_attach = {(u, v): p for u, v, p in nx.preferential_attachment(G, ebunch=pairs)}

    metrics = []
    for u, v in pairs:
        metrics.append(
            {
                "user1": u,
                "user2": v,
                "jaccard": jaccard[(u, v)],
                "adamic_adar": adamic_adar[(u, v)],
                "pref_attachment": pref_attach[(u, v)],
                "pagerank_1": pagerank[u],
                "pagerank_2": pagerank[v]
            }
        )

    return pd.DataFrame(metrics)

In [48]:
voting_data = pd.read_csv("data/votes_with_election_info.csv")
voting_data["start_time"] = pd.to_datetime(voting_data["start_time"])
voting_data.sort_values(
    by=["start_time"], inplace=True
)

## Computing Election-Specific Features

This section focuses on computing features for Wikipedia administrator elections. For each election:

1. **Time-Based Graph Construction**:
   - Creates a graph using data up to the election date
   - Filters interactions with weight ≥ 1 to focus on significant interactions

2. **Feature Extraction**:
   - Computes bipartite metrics between voters and the candidate
   - Generates social network metrics from the user projection
   - Combines both types of metrics into comprehensive feature vectors

3. **Output**:
   - Saves features for each election in separate CSV files
   - Files named with pattern: `article_edits.{date}.{candidate}.csv`

This temporal approach ensures we only use data that would have been available at the time of each election, preventing data leakage in subsequent analyses.

In [ ]:
missing_elections = voting_data[voting_data["start_time"] > "2005-08-31"]

for date in tqdm(missing_elections["start_time"].unique()):
    graph_data = create_graph_data(article_edits, cutoff_date=date)
    graph_data = graph_data[graph_data["weight"] >= 1]

    rows = voting_data[voting_data["start_time"] == date]
    voters = voting_data[voting_data["start_time"] == date]["voter"].unique()
    candidate = rows["candidate"].unique()[0]

    bipartite_graph: nx.DiGraph = bipartite_graph_from_edgelist(graph_data)
    bipartite_metrics = compute_user_article_metrics(
        bipartite_graph, voters, candidate
    )

    user_graph = bipartite.projected_graph(bipartite_graph, graph_data["user"].unique()).to_undirected()
    monopartite_metrics = compute_user_user_metrics(
    user_graph,
    voters,
    candidate
    )
    
    features = pd.merge(
        bipartite_metrics,
        monopartite_metrics,
        on=["user1", "user2"])
    
    if features.empty:
        continue

    features.to_csv(
        f"data/features/article_edits.{date.strftime(f'%Y-%m-%d')}.{candidate}.csv",
        index=False,
    )